<a href="https://colab.research.google.com/github/eswahn/stylegan2/blob/master/Stylegan2_RGBA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# INITIALIZE

%tensorflow_version 1.x

!git clone https://github.com/eswahn/stylegan2.git
%cd stylegan2

!nvcc test_nvcc.cu -o test_nvcc -run
!nvidia-smi -L

In [2]:
# LOAD DATASET

import math

dataset_name = 'your_dataset'
dataset_path = '/content/drive/My\ Drive/Data/%s.tar.gz' % dataset_name     # path to tgz archive
result_dir = '/content/drive/My\ Drive/Results/stylegan2/' + dataset_name   # path to results

resolution = 512
min_w, min_h = 4, 4                                                         # square
resolution_n = int(math.log2(resolution/min_w))                             # re resolution, see skyflynil repo


!cp $dataset_path ./
!tar -xf *.tar.gz
!rm *.tar.gz
!mkdir dataset

In [ ]:
# PREPARE DATASET

!python dataset_tool.py create_from_images ./dataset/$dataset_name $dataset_name

In [ ]:
# START / RESUME TRAINING

!python run_training.py --use-raw=false --config=config-e --metric=none --num-gpus=1 --total-kimg=20000 \
                        --min-h=$min_h --min-w=$min_w --res-log2=$resolution_n --mirror-augment=false --mirror-augment-v=false \
                        --result-dir=$result_dir --data-dir=./dataset --dataset=$dataset_name

In [ ]:
# TEST MODEL

import glob, os

network_dir = '/content/drive/My Drive/Results/stylegan2/' + dataset_name

allpickles = glob.glob(os.path.join(network_dir, '*', 'network-*.pkl'))
allpickles.sort(key=os.path.getctime)
pkl_path = '\"%s\"' % allpickles[-1]

!python run_generator.py generate-images --network=$pkl_path --seeds=0-100 --truncation-psi=1.0
